SHA 256 algorithm


https://qvault.io/cryptography/how-sha-2-works-step-by-step-sha-256/

https://www.researchgate.net/publication/259508311_A_compact_FPGA-based_processor_for_the_Secure_Hash_Algorithm_SHA-256 Outdated

https://github.com/xmrig/xmrig-proxy/wiki/API Seems like we are only able to use XMRig API to GET statistics about mining performance

As a result, may have to look elsewhere to implement custom algorithm.

https://xmrig.com/docs/miner/config/api#api-id

https://www.youtube.com/watch?v=ZhnJ1bkIWWk

https://medium.com/hackergirl/how-to-calculate-the-hash-of-a-block-in-bitcoin-8f6aebb0dc6d

https://analysis.null.place/how-do-the-bitcoin-mining-algorithms-work/#form4

Presentation:

https://analysis.null.place/how-do-the-bitcoin-mining-algorithms-work/#heading_block_hashing_in_detail

https://www.blockchain.com/btc/block/0

https://www.blockchain.com/explorer?view=btc

https://coinguides.org/x16r-algorithm-coins/

In [15]:
current_btc = 38317.74
JPYUSD = 115.47
print('BTC Price USD:', current_btc)
print('BTC Price Yen:', round(current_btc * JPYUSD, 2))

BTC Price USD: 38317.74
BTC Price Yen: 4424549.44


# Implementing SHA 256

https://qvault.io/cryptography/how-sha-2-works-step-by-step-sha-256/

In [65]:
# Step 0, convert string to binary
def convert_to_binary(test_str):
    return ' '.join(format(ord(i), '08b') for i in test_str)
convert_to_binary('hello world')

'01101000 01100101 01101100 01101100 01101111 00100000 01110111 01101111 01110010 01101100 01100100'

In [66]:
def int_to_64bit(num):
    # big endian
    # first separate into groups
    num = '{:064b}'.format(num)
    final_str = ""
    j = 0
    for i in range(71):
        if i%9 == 8:
            final_str += " "
        else:
            final_str += num[j]
            j += 1
    return final_str
int_to_64bit(88)

'00000000 00000000 00000000 00000000 00000000 00000000 00000000 01011000'

In [67]:
# Step 1, preprocessing
def preprocessing(binary_str):
    input_length = len(binary_str) - len(binary_str)//9
    binary_str += " 1"
    for i in range(len(binary_str), 503):
        if i % 9 == 8:
            binary_str += " "
        else:
            binary_str += '0'
    return binary_str + " " + int_to_64bit(input_length)
preprocessing(convert_to_binary('hello world'))

'01101000 01100101 01101100 01101100 01101111 00100000 01110111 01101111 01110010 01101100 01100100 10000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 01011000'

In [78]:
# step 2, initialize hash values
h0 = "0x6a09e667"
h1 = "0xbb67ae85"
h2 = "0x3c6ef372"
h3 = "0xa54ff53a"
h4 = "0x510e527f"
h5 = "0x9b05688c"
h6 = "0x1f83d9ab"
h7 = "0x5be0cd19"

In [84]:
# step 3, initialize more constants
constants = """0x428a2f98 0x71374491 0xb5c0fbcf 0xe9b5dba5 0x3956c25b 0x59f111f1 0x923f82a4 0xab1c5ed5
0xd807aa98 0x12835b01 0x243185be 0x550c7dc3 0x72be5d74 0x80deb1fe 0x9bdc06a7 0xc19bf174
0xe49b69c1 0xefbe4786 0x0fc19dc6 0x240ca1cc 0x2de92c6f 0x4a7484aa 0x5cb0a9dc 0x76f988da
0x983e5152 0xa831c66d 0xb00327c8 0xbf597fc7 0xc6e00bf3 0xd5a79147 0x06ca6351 0x14292967
0x27b70a85 0x2e1b2138 0x4d2c6dfc 0x53380d13 0x650a7354 0x766a0abb 0x81c2c92e 0x92722c85
0xa2bfe8a1 0xa81a664b 0xc24b8b70 0xc76c51a3 0xd192e819 0xd6990624 0xf40e3585 0x106aa070
0x19a4c116 0x1e376c08 0x2748774c 0x34b0bcb5 0x391c0cb3 0x4ed8aa4a 0x5b9cca4f 0x682e6ff3
0x748f82ee 0x78a5636f 0x84c87814 0x8cc70208 0x90befffa 0xa4506ceb 0xbef9a3f7 0xc67178f2"""
constants = constants.replace("\n", " ")
print(constants)

0x428a2f98 0x71374491 0xb5c0fbcf 0xe9b5dba5 0x3956c25b 0x59f111f1 0x923f82a4 0xab1c5ed5 0xd807aa98 0x12835b01 0x243185be 0x550c7dc3 0x72be5d74 0x80deb1fe 0x9bdc06a7 0xc19bf174 0xe49b69c1 0xefbe4786 0x0fc19dc6 0x240ca1cc 0x2de92c6f 0x4a7484aa 0x5cb0a9dc 0x76f988da 0x983e5152 0xa831c66d 0xb00327c8 0xbf597fc7 0xc6e00bf3 0xd5a79147 0x06ca6351 0x14292967 0x27b70a85 0x2e1b2138 0x4d2c6dfc 0x53380d13 0x650a7354 0x766a0abb 0x81c2c92e 0x92722c85 0xa2bfe8a1 0xa81a664b 0xc24b8b70 0xc76c51a3 0xd192e819 0xd6990624 0xf40e3585 0x106aa070 0x19a4c116 0x1e376c08 0x2748774c 0x34b0bcb5 0x391c0cb3 0x4ed8aa4a 0x5b9cca4f 0x682e6ff3 0x748f82ee 0x78a5636f 0x84c87814 0x8cc70208 0x90befffa 0xa4506ceb 0xbef9a3f7 0xc67178f2


# Using the algorithm for mining

In [17]:
# Sample SHA 256 encoding
import hashlib
a_string = 'hello world'
hashed_string = hashlib.sha256(a_string.encode('utf-8')).hexdigest()
print('Original String:', a_string)
print('Hash of string:', hashed_string)
print('Decimal value of hash:', int(hashed_string, 16))

Original String: hello world
Hash of string: b94d27b9934d3e08a52e52d7da7dabfac484efe37a5380ee9088f7ace2efcde9
Decimal value of hash: 83814198383102558219731078260892729932246618004265700685467928187377105751529


In [18]:
# Converts hex val to little endian
def to_little(val):
    if val[:2] == '0x': # handles formatting of hex()
        val = val[2:]
    little_hex = bytearray.fromhex(val)
    little_hex.reverse()

    str_little = ''.join(format(x, '02x') for x in little_hex)

    return str_little

In [19]:
# Converts and int to a little endian 4 byte hex value
# Used for version and nonce
def int_to_hex(num):
    num = '{:08x}'.format(num)
    return to_little(num)
print(int_to_hex(1))

01000000


In [20]:
# Generates the previous hash portion of the block header
def generate_prev_hash(test=True):
    if test:
        return '0000000000000000000000000000000000000000000000000000000000000000' # value for 1st bitcoin mined

In [21]:
#Generates the merkle root portion of the block header
def generate_merkle_root(test=True):
    if test:
        return to_little('4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b') # value for 1st bitcoin mined

In [22]:
# Generates the timestamp portion of block header
def generate_timestamp(test=True):
    if test:
        return to_little('495fab29') # hexadecimal timestamp of 1st bitcoin mined

In [23]:
# Generates the bits portion of the block header
def generate_bits(test=True):
    if test:
        return to_little('1d00ffff')

In [24]:
# Combines functions to create a the block header excluding the nonce
def generate_prefix():
    return int_to_hex(1) + generate_prev_hash() + generate_merkle_root() + generate_timestamp() + generate_bits()
generate_prefix()

'0100000000000000000000000000000000000000000000000000000000000000000000003ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a29ab5f49ffff001d'

In [25]:
def sha256(string):
    return hashlib.sha256(bytes.fromhex(string)).hexdigest()

In [26]:
# Test to make sure header is constructed correctly
string = generate_prefix() + int_to_hex(2083236893)
string == '0100000000000000000000000000000000000000000000000000000000000000000000003ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a29ab5f49ffff001d1dac2b7c'

True

In [27]:
def test_header(header):
    return to_little(sha256(sha256(header)))

In [29]:
# Iterates through nonce values until one produces a hash that is lower than the target
def find_nonce(prefix, target):
    i = 2080000000 # starting with a nonce close to the actual value
    header = ""
    while True:
        # status check
        if i %1000000 == 0:
            print(i)
        
        header = prefix + int_to_hex(i)
        attempt = test_header(header)
        if attempt < target:
            return attempt, header, i
        i += 1
target = '00000000ffff0000000000000000000000000000000000000000000000000000'
find_nonce(generate_prefix(), target)

2080000000
2081000000
2082000000
2083000000


('000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f',
 '0100000000000000000000000000000000000000000000000000000000000000000000003ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a29ab5f49ffff001d1dac2b7c',
 2083236893)